#Step 3) Linear Regression

## Imports

In [22]:
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly_resampler import FigureResampler, FigureWidgetResampler
from plotly.subplots import make_subplots
import pandas as pd
import polars as pl
import polars.selectors as cs
from datetime import date

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import cross_val_score, KFold
from sklearn.metrics import make_scorer, mean_squared_error, mean_absolute_error
import seaborn as sns

from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

%matplotlib inline

## Data

### Paths

In [23]:
Bornholm_Network_Manager = r"/home/adb/Documents/DTU/3rdTerm/MLES_Course/Bornholm Network Manager 23-09-2024 21-05-29.csv"
DMI_dataset = r"/home/adb/Documents/DTU/3rdTerm/MLES_Course/DMI Bornholm 24-09-2024 01-07-21.csv"
Energinet_dataset = r"/home/adb/Documents/DTU/3rdTerm/MLES_Course/Energiner Balancing Prices 24-09-2024 01-09-47.csv"
Forcast_norwegian_dataset = r"/home/adb/Documents/DTU/3rdTerm/MLES_Course/Norwegian Forecast dataset.csv"

### Name columns

In [24]:
time_cl = pl.col('time')
kalby_active_power = pl.col('804120')
#weather data
max_temp = pl.col('406464')
mean_temp = pl.col('406432')
min_temp = pl.col('406480')
accum_precip = pl.col('406656')
mean_wind_speed = pl.col('406640')
mean_wind_dirn = pl.col('406496')
mean_humidity = pl.col('406448')
#forecast data
fr_wind_dirn = pl.col('128270')
fr_accum_precip = pl.col('128238')
fr_mean_humidity = pl.col('128254')
fr_wind_speed = pl.col('128286')
fr_mean_temp = pl.col('128190')
#create column mappings
prev_day_power = pl.col('prev_day_power')

weekly_5th_quantile = pl.col("5thQuantile")
weekly_50th_quantile = pl.col("50thQuantile")
weekly_90th_quantile = pl.col("90thQuantile")

hourly_5th_quantile = pl.col("Hour_5thQuantile")
hourly_50th_quantile = pl.col("Hour_50thQuantile")
hourly_90th_quantile = pl.col("Hour_90thQuantile")

### Import Data nad Merging

In [25]:
wind_data = pl.read_csv(Bornholm_Network_Manager , separator= ',')

wind_data = wind_data.with_columns(
    pl.col('ts').str.to_datetime("%Y-%m-%d %H:%M:%S").alias('ts')
)
wind_data = wind_data.sort(pl.col('ts'))

simplified_columns = {}
new_col_wind = {} 
for c in wind_data.columns:
	new_name = c.split("|")[-1].strip()
	simplified_columns[new_name] = c
	new_col_wind[c] = new_name
 
wind_data = wind_data.rename(new_col_wind)
wind_data = wind_data.with_columns(
    pl.col("ts").dt.truncate("1h").alias("time")
)
wind_data = wind_data.group_by('time').agg(pl.all().mean())

In [26]:
dmi = pl.read_csv(DMI_dataset, separator= ',', ignore_errors= True)
dmi = dmi.select(
    pl.col('ts').str.to_datetime("%Y-%m-%d %H:%M:%S"),
    pl.exclude('ts')
)
dmi = dmi.sort(pl.col('ts'))

simplified_columns = {}
new_col_dmi = {} 
for c in dmi.columns:
	new_name = c.split("|")[-1].strip()
	simplified_columns[new_name] = c
	new_col_dmi[c] = new_name

dmi = dmi.rename(new_col_dmi)
dmi = dmi.with_columns(
    pl.col("ts").dt.truncate("1h").alias("time")
)
dmi = dmi.group_by('time').agg(pl.all().mean())

In [27]:
energinet = pl.read_csv(Energinet_dataset, separator=',', ignore_errors=True)

# Simplify column names
simplified_columns = {}
new_col_energinet = {}

for c in energinet.columns:
    new_name = c.split("|")[-1].strip()
    simplified_columns[new_name] = c
    new_col_energinet[c] = new_name

# Rename columns to their simplified names
energinet = energinet.rename(new_col_energinet)

# Ensure datetime-related columns are of string type
energinet = energinet.with_columns([
    pl.col('ts').cast(pl.Utf8),
    pl.col('804694').cast(pl.Utf8),
    pl.col('804695').cast(pl.Utf8)
])

# Convert the columns to datetime format
energinet = energinet.with_columns([
    pl.col('ts').str.to_datetime("%Y-%m-%d %H:%M:%S").alias('ts'),
    pl.col('804694').str.to_datetime("%Y-%m-%d %H:%M:%S").alias('804694'),
    pl.col('804695').str.to_datetime("%Y-%m-%d %H:%M:%S").alias('804695')
])

# Replace commas with periods for non-date columns and cast to Float64
# Cast non-date columns to string first to perform the replacement
non_date_columns = [col for col in energinet.columns if col not in ['ts', '804694', '804695', '804696']]

# Convert non-date columns to string to allow string operations
energinet = energinet.with_columns([
    pl.col(column).cast(pl.Utf8) for column in non_date_columns
])

# Perform string replacement and cast to Float64
energinet = energinet.with_columns([
    pl.col(column).str.replace(",", ".").cast(pl.Float64, strict=False) for column in non_date_columns
])

# Filter rows based on specific condition
energinet = energinet.filter(pl.col('804696') == 'DK2')

# Rename 'ts' column to 'time'
energinet = energinet.rename({'ts': 'time'})

# Truncate the 'time' column to hourly precision and add it back as a new column
energinet = energinet.with_columns(
    pl.col("time").dt.truncate("1h").alias("time")
)

# Group by 'time' and calculate the mean for all other columns
energinet = energinet.group_by('time').agg(pl.all().mean())

In [28]:
forecast = pl.read_csv(Forcast_norwegian_dataset , separator= ',', ignore_errors= True)

forecast = forecast.select(
    pl.col('ts').str.to_datetime("%Y-%m-%d %H:%M:%S"),
    pl.exclude('ts')
)
simplified_columns = {}
new_col_forecast = {} 
for c in forecast.columns:
	new_name = c.split("|")[-1].strip()
	simplified_columns[new_name] = c
	new_col_forecast[c] = new_name

forecast = forecast.rename(new_col_forecast)

forecast = forecast.rename({'ts':'time'})
forecast = forecast.with_columns(
    pl.col("time").dt.truncate("1h").alias("time")
)
forecast = forecast.group_by('time').agg(pl.all().mean())

In [29]:
# merge all datasets into one
temp1 = wind_data.join(dmi, on = 'time', how = 'left', validate= 'm:1')
cumulative_dataset = temp1.join(forecast, on = 'time', how = 'left', validate= 'm:1')
#cumulative_dataset = temp2.join(energinet, on='time', how = 'left', validate = 'm:1')
cumulative_dataset = cumulative_dataset.drop('ts', 'ts_right')
cumulative_dataset = cumulative_dataset.drop_nulls(subset = ['804120'])
cumulative_dataset = cumulative_dataset.with_columns(kalby_active_power.shift(24).alias('prev_day_power'))

### Adding Quantiles

In [30]:
#quantile of the week ahead of that data point
cumulative_dataset = cumulative_dataset.sort("time")
df = cumulative_dataset.rolling(index_column= 'time', period='1w').agg([
        pl.quantile("804120", 0.05).alias("5thQuantile"),
        pl.quantile("804120", 0.50).alias("50thQuantile"),
        pl.quantile("804120", 0.95).alias("90thQuantile"),
])
cumulative_dataset = cumulative_dataset.join(df, on="time", how="left")

In [31]:
#quantile of the hour of the day for the previous week
cumulative_dataset = cumulative_dataset.with_columns(pl.col("time").dt.hour().alias("hour_of_day"))
cumulative_dataset = cumulative_dataset.with_columns([
    pl.lit(None).cast(pl.Float64).alias("Hour_5thQuantile"),
    pl.lit(None).cast(pl.Float64).alias("Hour_50thQuantile"),
    pl.lit(None).cast(pl.Float64).alias("Hour_90thQuantile"),
])
rolling_quantile_hourly = pl.DataFrame()
for hour in range(24):  
    hour_df = cumulative_dataset.filter(pl.col("time").dt.hour() == hour)
    rolling_quantiles = (hour_df.rolling(index_column="time", period="1w").agg([
            pl.quantile("804120", 0.05).alias("Hour_5thQuantile"),
            pl.quantile("804120", 0.50).alias("Hour_50thQuantile"),
            pl.quantile("804120", 0.95).alias("Hour_90thQuantile"),
        ]))
    rolling_quantile_hourly = pl.concat([rolling_quantile_hourly, rolling_quantiles], how="vertical")
rolling_quantile_hourly = rolling_quantile_hourly.sort("time")

for quantile_col in ["Hour_5thQuantile", "Hour_50thQuantile", "Hour_90thQuantile"]:
    cumulative_dataset = cumulative_dataset.with_columns(
        pl.when(cumulative_dataset["time"].is_in(rolling_quantile_hourly["time"]))
          .then(rolling_quantile_hourly[quantile_col])
          .otherwise(pl.col(quantile_col)).alias(quantile_col)
    )

## Linear Regression - Gradien descent algorithm

### Linear Regression

In [50]:
cumulative_dataset

In [49]:
cumulative_dataset = cumulative_dataset.drop_nulls()

In [45]:
cumulative_dataset.drop_na()

Linear regression aims to model the relationship between a dependent variable (target = wind power production) y, and several independent variables (features) X. The goal is to find the best-fitting linear equation that predicts y based on X.

$$
y = \theta_0 + \theta_1 X_1 + \theta_2 X_2 + \ldots + \theta_n X_n
$$

theta 0 is the bias thetas are the weights

In [39]:
X = cumulative_dataset.drop(['804120']) # Features (excluding the target column)
y = cumulative_dataset['804120'].reshape(-1, 1) # Target variable

# Add a column of ones to include the bias term (intercept) in the calculation
X_b = np.c_[np.ones((X.shape[0], 1)), X] # X_b = [[1, x1], [1, x2], ..., [1, xn]]

In [40]:
y

It is basen on minimising the mean squared error

In [ ]:
def MSE(X, y, theta):
    predictions = X.dot(theta)
    errors = predictions - y
    return (1 / 2) * np.sum(errors ** 2)

In [ ]:
import polars as pl
import numpy as np
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

# Assuming cumulative_dataset is your Polars DataFrame

# Extract features and target from the Polars DataFrame
X = cumulative_dataset.drop('804120').to_numpy()  # Features (excluding the target column)
y = cumulative_dataset['804120'].to_numpy().reshape(-1, 1)  # Target variable reshaped to be a column vector

# Standardize the features
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Add a column of ones to include the bias term (intercept) in the calculation
X_b = np.c_[np.ones((X.shape[0], 1)), X]  # X_b = [[1, x1], [1, x2], ..., [1, xn]]

# Hyperparameters
learning_rate = 0.001  # Reduced learning rate
n_iterations = 1000    # Number of iterations
m = len(X_b)           # Number of samples

# Initialize weights with smaller random values
theta = np.random.randn(X_b.shape[1], 1) * 0.01  # Initialize theta for all features including the bias term

# Cost Function
def compute_cost(X, y, theta):
    predictions = X.dot(theta)
    errors = predictions - y
    return (1 / (2 * m)) * np.sum(errors ** 2)

# Gradient Descent Function
def gradient_descent(X, y, theta, learning_rate, n_iterations):
    cost_history = []
    
    for iteration in range(n_iterations):
        gradients = (1 / m) * X.T.dot(X.dot(theta) - y)
        theta = theta - learning_rate * gradients
        cost = compute_cost(X, y, theta)
        cost_history.append(cost)
        
        # Optional: print cost every 100 iterations for tracking
        if iteration % 100 == 0:
            print(f"Iteration {iteration}, Cost: {cost}")
    
    return theta, cost_history

# Train the model
theta_best, cost_history = gradient_descent(X_b, y, theta, learning_rate, n_iterations)

# Print the resulting parameters
print("Best-fit parameters (theta):", theta_best)



In [ ]:
# Plotting the cost history
plt.plot(range(n_iterations), cost_history)
plt.xlabel("Iterations")
plt.ylabel("Cost")
plt.title("Cost Function Reduction Over Time")
plt.show()
